In [1]:
%%sh
pwd

/mnt/m/Documents/DNN Final Project/spacecraft-pose-pose-estimation-runtime/juptyer-notebooks


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate, Flatten, Conv2D, MaxPooling2D, AvgPool2D, Lambda
from keras.optimizers import Adam
from keras import Model
import math
import tensorflow as tf
from keras.models import Sequential

2024-04-29 22:58:27.201764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 22:58:27.201894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 22:58:27.292485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 22:58:27.304571: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 22:58:29.050826: W tensorflow/compiler/tf2

In [3]:
dfY = pd.read_csv("../data/train_labels.csv")
df_range = pd.read_csv("../data/range.csv")
df_range = df_range.ffill()
dfY = pd.merge(dfY, df_range, left_on=['chain_id','i'], right_on=['chain_id','i'])
dfY = dfY.sample(n=5000, random_state=42)
dfY["path"] = [f"../data/images/{id}/{i:03d}.png" for id, i in zip(dfY["chain_id"], dfY["i"])]
dfY["base_path"] = [f"../data/images/{id}/000.png" for id, i in zip(dfY["chain_id"], dfY["i"])]
x_train, x_test, y_train, y_test = train_test_split(dfY[["path", "base_path", "range"]], dfY[["x", "y", "z", "qw","qx","qy","qz"]], test_size=0.5, random_state=42)
x_train = pd.DataFrame(x_train, columns=["path", "base_path", "range"])
print(x_train)
x_test = pd.DataFrame(x_test, columns=["path", "base_path", "range"])

                                    path                          base_path  \
59425  ../data/images/e8bd0b6fd9/025.png  ../data/images/e8bd0b6fd9/000.png   
25193  ../data/images/6397cf55d5/093.png  ../data/images/6397cf55d5/000.png   
46927  ../data/images/b932d56657/027.png  ../data/images/b932d56657/000.png   
64193  ../data/images/f7ca61cc93/093.png  ../data/images/f7ca61cc93/000.png   
3437   ../data/images/0e90967346/037.png  ../data/images/0e90967346/000.png   
...                                  ...                                ...   
51345  ../data/images/cad338efca/045.png  ../data/images/cad338efca/000.png   
12496  ../data/images/2db4b73861/096.png  ../data/images/2db4b73861/000.png   
45569  ../data/images/b44cf93326/069.png  ../data/images/b44cf93326/000.png   
34780  ../data/images/88b7f5fa26/080.png  ../data/images/88b7f5fa26/000.png   
21564  ../data/images/542f53a194/064.png  ../data/images/542f53a194/000.png   

       range  
59425  386.4  
25193  240.4  
46927 

In [4]:
#model_pos.save('../models/test_model-logcosh-c1x2-16x3-3e-000005-larger-kernels-7500-depth.keras')

In [5]:
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv')

# Loads all images found and trains given model
def load_images_and_train_model(model, is_pos_model):
    with tf.device('/GPU:0'):
        model.summary()
        
        #for i in range(batches):
            #batch = load_new_batch()
            #model.fit([np.asarray(batch[0]), np.asarray(batch[1])], y_train[100 * i : 100*(i+1)], epochs=40, batch_size=batch_size_b)
        if is_pos_model:
            ypos = pd.DataFrame(y_train, index=y_train.index, columns=["x", "y", "z"])
            model.fit(CustomDataGenerator(x_train, y_train, batch_size=100), epochs=3)
        else:
            yquat = pd.DataFrame(y_train, index=y_train.index, columns=["qw", "qx", "qy", "qz"])
            model.fit(CustomDataGenerator(x_train, yquat, batch_size=100), epochs=3)


        
    #     ypreddf = pd.DataFrame(ypred.T, index=image_label_ds.take(count * 100 * 0.2).index_array, columns=["x", "y", "z"])
    #     dfY = pd.read_csv("../data/train_labels.csv")[:count * 100]
    #     dfY_merged = pd.merge(ypreddf.drop(["x", "y", "z"], axis=1), dfY, left_index=True, right_index=True)
    #     dfYidx = dfY.drop(["x", "y", "z", "qw", "qx", "qy", "qz"], axis=1)
    #     df_merged = pd.merge(ypreddf, dfYidx, left_index=True, right_index=True)
    #     df_merged["qw"] = 0
    #     df_merged["qx"] = 0
    #     df_merged["qy"] = 0
    #     df_merged["qz"] = 0
    #     df_merged.loc[df_merged["i"] == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
    #     df_merged.set_index(["chain_id", "i"], inplace=True)
    #    # df_merged.drop(["id"], axis=1, inplace=True)
    #     ToCSV(df_merged, "cnn-test")
    #     ToCSV(dfY_merged, "cnn-true")

    return model

# from https://github.com/1988kramer/camera-pose/blob/master/camera-pose.py 
def create_conv_branch(input_shape):
	model = Sequential()
	model.add(Conv2D(96, kernel_size=(11,11),
					 strides=4, padding='valid',
					 activation='relu',
					 input_shape=input_shape))
	model.add(MaxPooling2D(pool_size=(2,2), strides=2))
	model.add(Conv2D(128, kernel_size=(7,7),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=1))
	model.add(Conv2D(256, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	# model.add(Conv2D(256, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(Conv2D(128, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(MaxPooling2D(pool_size=(3,3), strides=2))
	# model.add(Conv2D(256, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(Conv2D(128, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	return model

def create_conv_branch_pos(input_shape):
	model = Sequential()
	model.add(Conv2D(96, kernel_size=(11,11),
					 strides=4, padding='valid',
					 activation='relu',
					 input_shape=input_shape))
	model.add(MaxPooling2D(pool_size=(2,2), strides=2))
	model.add(Conv2D(128, kernel_size=(7,7),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=1))
	model.add(Conv2D(256, kernel_size=(5,5),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(256, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=2))
	model.add(Conv2D(256, kernel_size=(5,5),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	return model

In [6]:
# https://stackoverflow.com/questions/63827339/how-to-build-a-custom-data-generator-for-keras-tf-keras-where-x-images-are-being
class CustomDataGenerator(tf.keras.utils.Sequence):

    ''' Custom DataGenerator to load img 
    
    Arguments:
        data_frame = pandas data frame in filenames and labels format
        batch_size = divide data in batches
        shuffle = shuffle data before loading
        img_shape = image shape in (h, w, d) format
        augmentation = data augmentation to make model rebust to overfitting
    
    Output:
        Img: numpy array of image
        label : output label for image
    '''
    
    def __init__(self, data_frame, labels, batch_size=10, img_shape=None, augmentation=True, num_classes=None):
        self.data_frame = data_frame
        self.labels = labels
        self.train_len = len(data_frame)
        self.batch_size = batch_size
        self.img_shape = img_shape
        self.num_classes = num_classes
        print(f"Found {self.data_frame.shape[0]} images belonging to {self.num_classes} classes")

    def __len__(self):
        ''' return total number of batches '''
        #self.data_frame = shuffle(self.data_frame)
        return math.ceil(self.train_len/self.batch_size)

    def on_epoch_end(self):
        ''' shuffle data after every epoch '''
        # fix on epoch end it's not working, adding shuffle in len for alternative
        pass
    
    def __data_augmentation(self, img):
        ''' function for apply some data augmentation '''
        img = tf.image.resize(img, 512, 640)
        img = tf.image.random_flip_up_down(img)
        return img
        
    def __get_image(self, file_id):
        """ open image with file_id path and apply data augmentation """
        img = np.asarray(Image.open(file_id))
        img = np.resize(img, self.img_shape)
        img = self.__data_augmentation(img)

        return img

    def __get_label(self, label_id):
        """ uncomment the below line to convert label into categorical format """
        #label_id = tf.keras.utils.to_categorical(label_id, num_classes)
        return label_id

    def __getitem__(self, idx):
        # batch_x = self.data_frame["filenames"][idx * self.batch_size:(idx + 1) * self.batch_size]
        # batch_y = self.data_frame["labels"][idx * self.batch_size:(idx + 1) * self.batch_size]
        # # read your data here using the batch lists, batch_x and batch_y
        # x = [self.__get_image(file_id) for file_id in batch_x] 
        # y = [self.__get_label(label_id) for label_id in batch_y]

        y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        paths = self.data_frame["path"][idx * self.batch_size:(idx + 1) * self.batch_size]
        base_paths = self.data_frame["base_path"][idx * self.batch_size:(idx + 1) * self.batch_size]

        #raw = [tf.io.read_file(path) for path in paths]
        image = [tf.image.resize(tf.image.decode_png(tf.io.read_file(path), channels=3), (512,640)) for path in paths]
        #base_raw = tf.io.read_file(base_path)
        base_image = [tf.image.resize(tf.image.decode_png(tf.io.read_file(path), channels=3), (512,640)) for path in base_paths]

        # return [np.asarray(base_image), np.asarray(image), np.asarray(self.data_frame[idx * self.batch_size:(idx + 1) * self.batch_size]["range"])], y
        return [np.asarray(base_image), np.asarray(image), np.asarray(self.data_frame[idx * self.batch_size:(idx + 1) * self.batch_size]["range"])], y

        #return tf.convert_to_tensor(x), tf.convert_to_tensor(y)

In [7]:
def NormalizeQuat(x: tf.Tensor):
    norm = tf.sqrt(tf.reduce_sum(tf.square(x)))
    x = x / norm
    return x

In [8]:

input_shape=(512, 640, 3)

# Rotation model
# conv_branch = create_conv_branch(input_shape)
# branch_a = Input(shape=input_shape)
# branch_b = Input(shape=input_shape)

# processed_a = conv_branch(branch_a)
# processed_b = conv_branch(branch_b)

# concat = keras.layers.concatenate([processed_a, processed_b])
# c1 = Conv2D(256, kernel_size=(7,7),
# 					 strides=1, padding='same',
# 					 activation='relu')(concat)
# c1 = Conv2D(128, kernel_size=(3,3),
# 					 strides=1, padding='same',
# 					 activation='relu')(c1)
# c1 = GlobalAveragePooling2D()(c1)
# x1 = Flatten()(c1)
# x1 = Dropout(0.25)(x1)
# x1 = Dense(8)(x1)
# x1 = Dropout(0.25)(x1)
# output = Dense(4)(x1)
# postprocess = Lambda(NormalizeQuat, output_shape=(None,4))(output)

# model_rot = Model(inputs=[branch_a,branch_b], outputs=postprocess)
# model_rot.compile(optimizer=Adam(learning_rate=0.001), loss="log_cosh")
# model_rot.summary()
# load_images_and_train_model(model_rot, False)

# Position model
conv_branch = create_conv_branch_pos(input_shape)
branch_a = Input(shape=input_shape)
branch_b = Input(shape=input_shape)

processed_a = conv_branch(branch_a)
processed_b = conv_branch(branch_b)

concat = keras.layers.concatenate([processed_a, processed_b])
c1 = Conv2D(256, kernel_size=(7,7),
					 strides=1, padding='same',
					 activation='relu')(concat)
c1 = Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu')(c1)
# c1 = Conv2D(64, kernel_size=(3,3),
# 					 strides=1, padding='same',
# 					 activation='relu')(c1)
c1 = MaxPooling2D(pool_size=(2,2))(c1)
x1 = Flatten()(concat)
r1 = Input(shape=1)
concat2 = keras.layers.concatenate([x1,r1])
x1 = Dropout(0.25)(concat2)
x1 = Dense(16)(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(16)(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(16)(x1)
output = Dense(7)(x1)

model_pos = Model(inputs=[branch_a,branch_b,r1], outputs=output)
model_pos.compile(optimizer=Adam(learning_rate=0.00005), loss="log_cosh")
model_pos.summary()
load_images_and_train_model(model_pos, True)


2024-04-29 22:58:31.568224: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 22:58:31.663714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 22:58:31.664095: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 22:58:31.666677: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 22:58:31.667077: I external/local_xla/xla/stream_executor

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 640, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 512, 640, 3)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 30, 38, 128)          2341760   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 concatenate (Concatenate)   (None, 30, 38, 256)          0         ['sequential[0][0]',      

2024-04-29 22:58:50.379673: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-29 22:58:50.640511: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-29 22:58:51.081705: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.71GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-29 22:58:54.721407: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.71GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-29 22:59:12.701332: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn chil

25/25 [==============================] - 269s 9s/step - loss: 61.7006
Epoch 2/3
25/25 [==============================] - 215s 9s/step - loss: 61.3034
Epoch 3/3
25/25 [==============================] - 213s 9s/step - loss: 61.2684


In [9]:
#model_pos = keras.models.load_model('../models/test_model-logcosh-16x3-3e-00001-larger-kernels.keras')
#model_pos = keras.models.load_model('../models/test_model-logcosh-c1x2-16x3-3e-000005-larger-kernels-7500-depth.keras')

In [10]:
# Make predictions
#ypred = pd.DataFrame(model_rot.predict(CustomDataGenerator(x_test, y_test, 20)), index=y_test.index, columns=["qw", "qx", "qy", "qz"]) # Predict rotation
ypred = pd.DataFrame(model_pos.predict(CustomDataGenerator(x_test, y_test, 50)), index=y_test.index, columns=["x", "y", "z", "qw", "qx", "qy", "qz"]) # Predict position
#ypred = pd.merge(ypred_pos, ypred_rot, left_index=True, right_index=True)
#ypred[["qw", "qx", "qy", "qz"]] = ypred[["qw", "qx", "qy", "qz"]].clip(lower=-1, upper=1)

ypred["chain_id"] = dfY.loc[ypred.index].chain_id
ypred["i"] = dfY.loc[ypred.index].i.astype(int)
ypred.set_index(["chain_id", "i"], inplace=True)
y_pred_norm = np.linalg.norm(ypred[["qw", "qx", "qy", "qz"]], axis=1)
ypred["qw"] = ypred["qw"] / y_pred_norm
ypred["qx"] = ypred["qx"] / y_pred_norm
ypred["qy"] = ypred["qy"] / y_pred_norm
ypred["qz"] = ypred["qz"] / y_pred_norm
#ypred.loc[ypred.i == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
# ypred[["qw", "qx", "qy", "qz"]] = [0,0,0,0]
#ypred[["x", "y", "z"]] = [0,0,0]
#ypred = ypred[["chain_id", "i", "x", "y", "z", "qw", "qx", "qy", "qz"]]
#ypred.loc[ypred.i == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
ypred.loc[(slice(None), 0), ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
#ypred.set_index(["chain_id", "i"], inplace=True)
ToCSV(ypred, "cnn-test")
y_copy = y_test.copy()
y_copy["chain_id"] = dfY.loc[y_copy.index].chain_id
y_copy["i"] = dfY.loc[y_copy.index].i.astype(int)
y_copy.set_index(["chain_id", "i"], inplace=True)
ToCSV(y_copy, "cnn-true")

#model.save('../models/test_model.keras')

Found 2500 images belonging to None classes
50/50 [==============================] - 221s 4s/step
